In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

C:\Users\swdtk\AppData\Local\Temp\ipykernel_6280\2721483381.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [10]:
def getRateDataFrame(codeStr):
    """
    関数の説明：
        codeStrで指定したレート一覧を、ヤフーファイナンスから取得する関数
    
    引数：
        codeStr:ヤフーファイナンスから取得する通貨名を指定

    戻値：
        rateDf:OHCLが記載されたDataFrame
    """    
    #ヤフーファイナンスからレートを取得
    rateDf = yf.download(codeStr, start='2004-01-01', end='2040-12-31')

    #不要な列の削除
    del rateDf['Adj Close']
    del rateDf['Volume']

    #indexを列に変換
    rateDf.reset_index(inplace= True)
    
    #parquetとして保存
    fileName =  './csv/' + codeStr + '.csv'
    rateDf.to_csv(fileName, encoding='cp932', index=False)

    return rateDf


def readRateDataFrame(codeStr):
    """
    関数の説明：
        getRateDataFrameでダウンロードしたCSVファイルを読み取る関数
    
    引数：
        codeStr:ヤフーファイナンスから取得する通貨名を指定

    戻値：
        rateDf:OHCLが記載されたDataFrame
    """
    codeValue = getRateCodeDic[codeStr]
    fileName =  './csv/' + codeValue + '.csv'
    rateDf = pd.read_csv(fileName, encoding='cp932', parse_dates=[0])

    return rateDf
    


def makeHendoRitu(rateDf, nRaw):
    """
    関数の説明：
        n行前の始値と、現在の始値の変動率を記載する
    
    引数：
        rateDf:Date、Open、High、Low、Closeで構成されるレートの一覧
        nRaw:どの期間の変動率を取得するか（1なら1行前）

    戻値：
        rateDf:変動率を記載したDataFrame
    """
    #n行前の始値と、現在の始値の変動率を記載
    nDaysStr = str(nRaw) + 'period'
    rateDf[nDaysStr] = rateDf['Open'].shift(nRaw)
    rateDf['変動率'] = (( rateDf['Open'] / rateDf[nDaysStr] ) -1 ) * 100
    del rateDf[nDaysStr]

    return rateDf

def makePeriodDataFrame(nRaw, readFlg=0):
    """
    関数の説明：
        各通貨の指定した期間の変動率を１つのDataFrameにまとめる関数
    
    引数：
        nRaw:指定した期間
        readFlg:
            0:保存済みのCSVファイルを読み込む
            1:新規にヤフーファイナンスからダウンロードする

    戻値：
        merge_Df:各通貨の指定した期間の変動率をまとめたDataFrame
    """
    merge_Df = pd.DataFrame()

    for codeKey in currencyList:
        #レート取得
        codeValue = getRateCodeDic[codeKey]
        if readFlg == 0:
            rateDf = readRateDataFrame(codeStr=codeKey)
        elif readFlg == 1: 
            rateDf = getRateDataFrame(codeStr=codeValue)

        #各通貨の指定した期間のDataFrameを取得する
        rateDf = makeHendoRitu(rateDf, nRaw=nRaw)
        rateDf = rateDf[['Date', '変動率']] #必要な列のみにする
        rateDf = rateDf.rename(columns={'変動率':codeKey}) #列名を通貨名に変更

        #取得してきた各通貨の変動率をマージしていく
        if len(merge_Df) == 0:
            merge_Df = rateDf
        else:
            merge_Df = pd.merge(merge_Df, rateDf, how='left', on='Date')
    
    fileName = str(nRaw) + '_hendoritsu.csv'
    merge_Df.to_csv('./csv/' + fileName, encoding='cp932', index=False)
    
    return merge_Df


def mergeZenjitsuhi(readFlg=0):
    """
    関数の説明：
        各通貨の前日比を計算してマージする
    
    引数：
        readFlg:
            0:保存済みのCSVファイルを読み込む
            1:新規にヤフーファイナンスからダウンロードする

    戻値：
        mergeZenjitsu_Df:マージしたDataFrame
    """
    mergeZenjitsu_Df = pd.DataFrame()

    for codeKey in currencyList:
        #レート取得
        codeValue = getRateCodeDic[codeKey]
        if readFlg == 0:
            rateDf = readRateDataFrame(codeStr=codeKey)
        elif readFlg == 1: 
            rateDf = getRateDataFrame(codeStr=codeValue)

        #各通貨の指定した期間のDataFrameを取得する
        rateDf['翌日の始値'] = rateDf['Open'].shift(-1)
        rateDf['翌日の始値比'] = ((rateDf['翌日の始値'] / rateDf['Open']) - 1) *100
        rateDf = rateDf[['Date', '翌日の始値比']] #必要な列のみにする
        rateDf = rateDf.rename(columns={'翌日の始値比':codeKey}) #列名を通貨名に変更

        #取得してきた各通貨の変動率をマージしていく
        if len(mergeZenjitsu_Df) == 0:
            mergeZenjitsu_Df = rateDf
        else:
            mergeZenjitsu_Df = pd.merge(mergeZenjitsu_Df, rateDf, how='left', on='Date')
    
    return mergeZenjitsu_Df


def getMaxMin(df):
    """
    関数の説明：
        makePeriodDataFrameで作成したDataFrameの最大列名と最小列名を取得する
    
    引数：
        df:makePeriodDataFrameで作成したDataFrame

    戻値：
        df:最大列名と最小列名を取得したDataFrame
    """
    # 最大値を持つ列を特定
    df['MAX列'] = df.apply(lambda row: row[currencyList].idxmax(), axis=1)

    # 最小値を持つ列を特定
    df['MIN列'] = df.apply(lambda row: row[currencyList].idxmin(), axis=1)
    
    return df

def mergeMaxMinResult(x,y,z):
    """
    関数の説明：
        getMaxMinで作成した3つの変動率を作成し、それをマージする関数
    
    引数：
        x=1つ目の期間
        y=2つ目の期間
        z=3つ目の期間

    戻値：
        merge_Df:3つの結果をマージしたDataFrame
    """
    merge_Df = pd.DataFrame()

    #指定した期間の変動率をまとめる
    part1Df = makePeriodDataFrame(nRaw=x, readFlg=0)
    #どの通貨が最大と最小かを求める
    part1Df = getMaxMin(df=part1Df)
    part1Df = part1Df[['Date','MAX列','MIN列']]

    #指定した期間の変動率をまとめる
    part2Df = makePeriodDataFrame(nRaw=y, readFlg=0)
    #どの通貨が最大と最小かを求める
    part2Df = getMaxMin(df=part2Df)
    part2Df = part2Df[['Date','MAX列','MIN列']]

    #指定した期間の変動率をまとめる
    part3Df = makePeriodDataFrame(nRaw=z, readFlg=0)
    #どの通貨が最大と最小かを求める
    part3Df = getMaxMin(df=part3Df)
    part3Df = part3Df[['Date','MAX列','MIN列']]


    merge_Df = pd.merge(part1Df, part2Df, how='left', on='Date')
    merge_Df = pd.merge(merge_Df, part3Df, how='left', on='Date')    
    
    return merge_Df

def selectCondition(type, merge_Df):
    """
    関数の説明：
        買いシグナルと売りシグナルのロジックを選択する関数
    
    引数：
        type:
            1:3つの期間がすべて一致したらTrue
            2:2つの期間が一致したらTure
            3:1つの期間しか見ない
        merge_df:ロジック計算対象のDataFrame

    戻値：
        conditionBuy:買いシグナル
        conditionSell:売りシグナル
    """
    if type == 1:
        conditionBuy = (merge_Df['MAX列_x']==merge_Df['MAX列_y']) & ( merge_Df['MAX列_x']==merge_Df['MAX列'])
        conditionSell = (merge_Df['MIN列_x']==merge_Df['MIN列_y']) & ( merge_Df['MIN列_x']==merge_Df['MIN列'])
    
    elif type == 2:
        conditionBuy = (merge_Df['MAX列_x']==merge_Df['MAX列_y'])
        conditionSell = (merge_Df['MIN列_x']==merge_Df['MIN列_y'])
        
    elif type == 3:
        conditionBuy = (merge_Df['MAX列_x'].notnull())
        conditionSell = (merge_Df['MIN列_x'].notnull())
    
    
    return conditionBuy, conditionSell



def make_result_dataframe(readFlg, stX, stY, stZ, typeCode):
    #それぞれの期間を文字列にまとめる
    stMerge = str(stX) + '_' + str(stY) + '_' +  str(stZ)
    
    #各期間の最大最小をまとめる
    merge_Df = mergeMaxMinResult(x=stX, y=stY, z=stZ)

    #売買ロジックを選択する
    conditionBuy, conditionSell = selectCondition(type=typeCode, merge_Df=merge_Df)

    #それぞれの期間が一致したらフラグを立てる
    merge_Df['BuyFlg']=0
    merge_Df.loc[conditionBuy, 'BuyFlg'] = merge_Df['MAX列_x']

    merge_Df['SellFlg']=0
    merge_Df.loc[conditionSell, 'SellFlg'] = merge_Df['MIN列_x']

    #各通貨の現在のOpneと翌日のOpneとの比較結果をまとめる
    mergeZenjitsu_Df = mergeZenjitsuhi(readFlg=readFlg)

    #各通貨の変動率をマージ
    hendoritsuDf = pd.read_csv('./csv/1_hendoritsu.csv', encoding='cp932', parse_dates=[0])
    hendoritsuDic = {'USD':'USD変動率','EUR':'EUR変動率', 'GBP':'GBP変動率', 'NZD':'NZD変動率', 'AUD':'AUD変動率'}
    hendoritsuDf = hendoritsuDf.rename(columns=hendoritsuDic)
    merge_Df = pd.merge(merge_Df, hendoritsuDf, on='Date', how='left')
    value_list = list(hendoritsuDic.values())
    merge_Df['変動率平均'] = merge_Df[value_list].mean(axis=1)

    #merge_DfとmergeZenjitsu_Dfをマージ
    result_df = pd.merge(merge_Df, mergeZenjitsu_Df, on='Date', how='left')

    #結果をまとめる
    result_df['結果_Buy'] = 0
    result_df['結果_Sell'] = 0
    for codeKey in currencyList:
        condition = (result_df['BuyFlg']==codeKey)
        result_df.loc[condition, '結果_Buy'] = result_df[codeKey]
        
        condition = (result_df['SellFlg']==codeKey)
        result_df.loc[condition, '結果_Sell'] = result_df[codeKey] * -1
    result_df['type_code'] = typeCode

    #集計結果の合計値(買い)
    gpBuyResultdf = result_df.groupby(['BuyFlg'])['結果_Buy'].sum().reset_index()
    gpBuyResultdf = gpBuyResultdf[gpBuyResultdf['結果_Buy']!=0]

    #集計結果の合計値(売り)
    gpSellResultdf = result_df.groupby(['SellFlg'])['結果_Sell'].sum().reset_index()
    gpSellResultdf = gpSellResultdf[gpSellResultdf['結果_Sell']!=0]

    #合計値のDataFrameをマージ
    resultMerge_Df = pd.merge(gpBuyResultdf, gpSellResultdf, how='left', left_on='BuyFlg', right_on='SellFlg')
    resultMerge_Df['シミュレーションステータス']=stMerge + '_type:' + str(typeCode)

    #CSV保存
    result_df.to_csv(stMerge +'_result_df.csv', encoding='cp932', index=False)
    resultMerge_Df.to_csv('resultMerge_Df.csv', encoding='cp932', index=False)
    
    return resultMerge_Df

In [14]:
#取得する通貨
currencyList = ['USD', 'EUR', 'GBP', 'NZD', 'AUD']

#取得してくるコード一覧（すべてクロス円）
getRateCodeDic = {'USD':'USDJPY=X','EUR':'EURJPY=X', 'GBP':'GBPJPY=X', 'NZD':'NZDJPY=X', 'AUD':'AUDJPY=X'}

readFlg = 0 #ダウンロードするか腹持ちデータをみるか
stX, stY, stZ = 3,5,25 #期間
accumulatedDf  = pd.DataFrame()
typeCode = 1 #ロジックの選択

for stX in range(1, 10):
    resultMerge_Df = make_result_dataframe(readFlg=readFlg, stX=stX, stY=stY, stZ=stZ, typeCode=typeCode)

    if len(accumulatedDf)==0:
        accumulatedDf = resultMerge_Df
    else:
        accumulatedDf = pd.concat([accumulatedDf, resultMerge_Df], ignore_index=True)
    
    accumulatedDf.to_csv('accumulatedDf.csv', encoding='cp932', index=False)

C:\Users\swdtk\AppData\Local\Temp\ipykernel_6280\419163852.py:159: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  df['MAX列'] = df.apply(lambda row: row[currencyList].idxmax(), axis=1)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_6280\419163852.py:159: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  df['MAX列'] = df.apply(lambda row: row[currencyList].idxmax(), axis=1)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_6280\419163852.py:159: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  df['MAX列'] = df.apply(lambda row: row[currencyList].idxmax(), axis=1)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_6280\419163852.py:159: FutureWarning: The behavior of Series.idxmax with a

In [13]:
del accumulatedDf

In [16]:
result_df

,Date,MAX列_x,MIN列_x,MAX列_y,MIN列_y,MAX列,MIN列,BuyFlg,SellFlg,USD変動率,...,NZD変動率,AUD変動率,USD,EUR,GBP,NZD,AUD,結果_Buy,結果_Sell,type_code
0,2004-01-01,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,...,NaN,NaN,0.345959,-0.125775,-0.239541,-0.303770,0.000000,0.000000,0.0,1
1,2004-01-02,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.345959,...,-0.303770,0.000000,-0.363399,0.081488,0.198355,0.726147,0.956007,0.000000,0.0,1
2,2004-01-05,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-0.363399,...,0.726147,0.956007,-0.645284,-0.236867,0.031257,0.616322,0.567480,0.000000,0.0,1
3,2004-01-06,AUD,USD,NaN,NaN,NaN,NaN,0,0,-0.645284,...,0.616322,0.567480,-0.018822,0.259687,0.703092,0.064622,0.147207,0.000000,0.0,1
4,2004-01-07,AUD,USD,NaN,NaN,NaN,NaN,0,0,-0.018822,...,0.064622,0.147207,-0.028242,-0.710432,-0.206865,-0.103893,-0.183735,0.000000,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5219,2024-02-13,NZD,EUR,NZD,USD,USD,AUD,0,0,0.159547,...,-0.163598,0.223996,0.882835,0.287706,0.582093,-0.273108,-0.285005,0.000000,0.0,1
5220,2024-02-14,USD,NZD,USD,AUD,USD,AUD,USD,0,0.882835,...,-0.273108,-0.285005,-0.119419,0.105315,-0.281452,0.423930,0.577805,-0.119419,0.0,1
5221,2024-02-15,USD,NZD,USD,GBP,USD,AUD,USD,0,-0.119419,...,0.423930,0.577805,-0.397231,-0.016075,-0.160681,-0.059994,0.017384,-0.397231,0.0,1
5222,2024-02-16,EUR,NZD,AUD,GBP,USD,AUD,0,0,-0.397231,...,-0.059994,0.017384,0.152724,0.172723,0.203285,0.420203,0.283107,0.000000,0.0,1


In [4]:
conditionBuy, conditionSell = selectCondition(type=1)

In [6]:
conditionSell

0       False
1       False
2       False
3       False
4       False
        ...  
5219    False
5220     True
5221    False
5222    False
5223    False
Length: 5224, dtype: bool

In [7]:
merge_Df['MAX列_x'].notnull()

0       False
1        True
2        True
3        True
4        True
        ...  
5219     True
5220     True
5221     True
5222     True
5223     True
Name: MAX列_x, Length: 5224, dtype: bool

In [33]:
def makeSma(dataDf, tankiSma, choukiSma):
    """
    関数の説明：
        yfinanceで取得してきた株価のDataFrameを基に、
        ゴールデンクロスなら買い、デッドクロスなら決済のフラグを作成してくれる関数。
    
    Args:
        dataDf: yfinanceで取得してきた株価のDataFrame
        tankiSma: 短期移動平均線の日数
        choukiSma: 長期移動平均線の日数

    Returns:
        tradesDf: 売買フラグを付与したDataFrameを返す。
    """
    # 短期移動平均線を計算
    dataDf['短期SMA'] = dataDf['Adj Close'].rolling(window=tankiSma).mean()

    # 長期移動平均線を計算
    dataDf['長期SMA'] = dataDf['Adj Close'].rolling(window=choukiSma).mean()

    # ポジションを保持するための変数
    position = 0

    #ポジションをとった時のPrice
    numPositon = 0    

    # 取引履歴を保存するDataFrame
    tradesDf = pd.DataFrame(columns=['Date', 'Position', 'Price'])

    # 移動平均線を用いたバックテストを実施
    for i in range(len(dataDf)):
        # 短期移動平均線が長期移動平均線を上抜けした場合、買いポジションを取る
        if dataDf['短期SMA'][i] > dataDf['長期SMA'][i] and position == 0:
            position = 1
            tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)

        # 短期移動平均線が長期移動平均線を下抜けした場合、決済
        elif dataDf['短期SMA'][i] < dataDf['長期SMA'][i] and position == 1:
            position = 0
            tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Sell', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
    
    #決済金額を追記
    tradesDf['result']=0

    for i in range(len(tradesDf)):
        #Buyなら買う
        if tradesDf.iloc[i, 1] == 'Buy':
            numPositon = tradesDf.iloc[i, 2]
        #Sellなら決済
        if tradesDf.iloc[i, 1] == 'Sell':
            tradesDf.iloc[i, 3] = (tradesDf.iloc[i, 2] - numPositon) / numPositon

    return tradesDf



In [6]:
n225_sma = makeSma(dataDf=n225_train, tankiSma=5, choukiSma=25)

C:\Users\swdtk\AppData\Local\Temp\ipykernel_13084\714882119.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_13084\714882119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Sell', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_13084\714882119.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\App

          Date Position         Price    result
0   2000-02-08      Buy  19868.880859  0.000000
1   2000-03-13     Sell  19189.929688 -0.034172
2   2000-03-24      Buy  19958.080078  0.000000
3   2000-04-18     Sell  18969.519531 -0.049532
4   2000-06-08      Buy  17004.339844  0.000000
..         ...      ...           ...       ...
235 2018-10-12     Sell  22694.660156 -0.005550
236 2018-11-12      Buy  22269.880859  0.000000
237 2018-11-15     Sell  21803.619141 -0.020937
238 2018-11-29      Buy  22262.599609  0.000000
239 2018-12-07     Sell  21678.679688 -0.026229

[240 rows x 4 columns]


In [7]:
n225_sma['result'].sum()

0.5362064095993865

In [12]:
columns = ['短期SMA', '長期SMA', '結果']
resultDf = pd.DataFrame(columns=columns)

In [35]:
resultDf.to_csv('result3.csv', index=False, encoding='cp932')



In [40]:
import warnings
warnings.filterwarnings('ignore')

#短期線
for i in range(51, 250):

    #長期線    
    for j in range(i+1, 300):
        
        n225_sma = makeSma(dataDf=n225_train, tankiSma=i, choukiSma=j)

        # 結果を保存
        new_row = {'短期SMA': i, '長期SMA': j, '結果': n225_sma['result'].sum()}
        resultDf = resultDf.append(new_row, ignore_index=True)
        
resultDf.to_csv('result.csv', index=False, encoding='cp932')

In [36]:
# 日経225のデータをyfinanceから取得
n225_test = yf.download('^N225', start='2019-01-01', end='2023-12-31')

# テストデータで検証
n225_sma = makeSma(dataDf=n225_test, tankiSma=195, choukiSma=296)

[*********************100%***********************]  1 of 1 completed


In [37]:
n225_sma['result'].sum()

0.2498771142944017

In [26]:
n225_sma.to_csv('test.csv')